## Information Gain

- Entropy 함수를 도입하여 branch splitting
- Information Gain : Entropy를 사용하여 속성별 분류시 Impurity(복잡도)를 측정하는 지표
- (전체 Entropy - 속성별 Entropy)로 속성별 Information Gain을 계산함
- 속성별 Entropy가 증가할수록 Impurity(복잡도)가 증가한다
- 속성별 Entropy가 낮다는 것은 명확하게 데이터를 나눠줄 수 있다는 것을 의미한다
- 따라서 속성별 Entropy가 낮은 것을 attribute로 선택해서 branch를 나눈다

In [22]:
import pandas as pd
import numpy as np

In [23]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1)
pd_data

,RID,age,income,student,credit_rating,class_buys_computer
0,1,youth,high,no,fair,no
1,2,youth,high,no,excellent,no
2,3,middle_aged,high,no,fair,yes
3,4,senior,medium,no,fair,yes
4,5,senior,low,yes,fair,yes
5,6,senior,low,yes,excellent,no
6,7,middle_aged,low,yes,excellent,yes
7,8,youth,medium,no,fair,no
8,9,youth,low,yes,fair,yes
9,10,senior,medium,yes,fair,yes


In [24]:
buy = pd_data.loc[pd_data['class_buys_computer'] == 'yes']
not_buy = pd_data.loc[pd_data['class_buys_computer'] == 'no']

In [25]:
x = np.array([len(buy) / len(pd_data) , len(not_buy) / len(pd_data)])
x

array([0.64285714, 0.35714286])

In [26]:
y = np.log2(x)
y

array([-0.63742992, -1.48542683])

In [27]:
x * y

array([-0.40977638, -0.53050958])

In [28]:
# Entropy
info_all = -sum(x * y)
info_all

0.9402859586706311

In [59]:
# 위의 Entropy 구하는 과정을 함수화
def get_info(df):
    buy = df.loc[df["class_buys_computer"]=="yes"]
    not_buy = df.loc[df["class_buys_computer"]=="no"]
    x = np.array([len(buy)/len(df),len(not_buy)/len(df)])
    y = np.log2(x[x!=0])
    

    info_all = - sum(x[x!=0] * y)
    return info_all

In [60]:
get_info(pd_data)

0.9402859586706311

In [70]:
# 각각의 attribute별로 구하기:
def get_attribute_info(df, attribute_name):
    attribute_values = pd_data[attribute_name].unique()
    get_infos = []
    for value in attribute_values:
        split_df = pd_data.loc[pd_data[attribute_name] == value]
        
        get_infos.append((len(split_df) / len(df)) * get_info(split_df))
    
    return sum(get_infos)

In [62]:
youth = pd_data.loc[pd_data['age'] == "youth"]
senior = pd_data.loc[pd_data['age'] == "senior"]
middle_aged = pd_data.loc[pd_data['age'] == "middle_aged"]

In [63]:
youth

,RID,age,income,student,credit_rating,class_buys_computer
0,1,youth,high,no,fair,no
1,2,youth,high,no,excellent,no
7,8,youth,medium,no,fair,no
8,9,youth,low,yes,fair,yes
10,11,youth,medium,yes,excellent,yes


In [68]:
get_info(youth), get_info(senior)

(0.9709505944546686, 0.9709505944546686)

In [67]:
get_info(middle_aged)

-0.0

#### 아래의 결과를 보면, age를 기준으로 했을 때 information gain이 가장 컸다.

- 뒤에 있는 attribute의 entropy가 작다
- 데이터를 분류할 때 명확하게 분류한다
- 따라서 Information Gain이 큰 것을 선택해서 데이터를 나눈다

In [71]:
# Gain(age) = 전체 information gain - 해당 attribute의 information gain값
get_info(pd_data) - get_attribute_info(pd_data, 'age')

0.24674981977443933

In [72]:
# income
get_info(pd_data) - get_attribute_info(pd_data, 'income')

0.02922256565895487

In [73]:
# student
get_info(pd_data) - get_attribute_info(pd_data, 'student')

0.15183550136234159

In [74]:
# credit
get_info(pd_data) - get_attribute_info(pd_data, 'credit_rating')

0.04812703040826949

In [75]:
# youth 인 것만 뽑아서 youth에 저장
youth = pd_data.loc[pd_data['age']=='youth']
youth

,RID,age,income,student,credit_rating,class_buys_computer
0,1,youth,high,no,fair,no
1,2,youth,high,no,excellent,no
7,8,youth,medium,no,fair,no
8,9,youth,low,yes,fair,yes
10,11,youth,medium,yes,excellent,yes


#### age는 이미 썼으므로, 나머지 3개 중에서 Information Gain이 가장 큰 것을 선택한다

- student가 가장 크므로, student를 기준으로 나눈다

In [76]:
get_info(youth) - get_attribute_info(youth, 'income')

-1.580026905978025

In [77]:
get_info(youth) - get_attribute_info(youth, 'student')

-1.2367106860085422

In [78]:
get_info(youth) - get_attribute_info(youth, 'credit_rating')

-1.527094404679944